In [35]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import copy

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks

Date Reader Challange Polimi 2017
Playlists created..
Tracks created..
Target playlists imported..
Target playlists created..
Target tracks created..
Train and URM created..
Attributes tracks created..
Split dataset in train-test, update target tracks and urm..
Saving data in ./DumpData/DataReaderTrainTestDump
Dataset train-test created


In [36]:
# album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender()
model.fit(icm)
s = model.similarity
r_al_ar = urm*s.T #or s
r_al_ar = normalize(r_al_ar, axis=0)
if evaluation: score = dr.evaluateMAP(r_al_ar)

# tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
icm = normalize(icm,axis=1)   # little improvment? or just overfitting?
model = BM25Recommender()
model.fit(icm)
s = model.similarity
r_tag = urm*s #######wuuuut no urm
r_tag = normalize(r_tag, axis=0)
if evaluation: score = dr.evaluateMAP(r_tag)
    
r_cb = r_al_ar + 0.3*r_tag

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0778
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0372


In [37]:
#collaborative filtering item based
m_cf_ib = CF_IB_asymmetric_cosine(urm, k=200, alpha=0.30, q=1, m=0)
s = m_cf_ib.s
r_cfib = urm*s

if evaluation: score = dr.evaluateMAP(r_cfib)

Start asymmetric cosine item-based model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model asymmetric cosine item-based done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0732


In [38]:
#
#
#
#
#

In [39]:
maps1, n1 = dr.evaluateMAP(r_cb,allmaps=True)
maps2, n2 = dr.evaluateMAP(r_cfib,allmaps=True)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0796
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0732


In [41]:
count = 0 
p_cb = {}

th = 0.20
for row in maps1:
    if(row[1]>=th):
        count = count +1
        p_cb[row[0]] = [row[1],row[2]]

count2 = 0 
p_cf_ib = {}
for row in maps2:
    if(row[1]>=th):
        count2 = count2 +1
        p_cf_ib[row[0]] = [row[1],row[2]]

print "cb: " + str(count)
print "cf: " + str(count2)

cb: 1672
cf: 1587


In [42]:
join = 0 
th2 = 0.15
for p in p_cb.keys():
    if p in p_cf_ib.keys():
        join = join +1
        m_cb = p_cb[p][0]
        m_cf = p_cf_ib[p][0]
        if m_cb - m_cf >th2:
            p_cf_ib.pop(p, None)
        elif m_cf - m_cb > th2:
            p_cb.pop(p,None)
        else:
            p_cf_ib.pop(p, None)
            p_cb.pop(p,None)
print "common: " + str(join)
print "new cb: " + str(len(p_cb))
print "new cf: " + str(len (p_cf_ib))

common: 669
new cb: 1194
new cf: 1076


In [43]:
join = 0
for p in p_cb.keys():
    if p not in p_cb_old_s1.keys():
        join = join +1
        p_cb.pop(p,None)
print join

join = 0
for p in p_cf_ib.keys():
    if p not in p_cf_ib_old_s1.keys():
        join = join +1
        p_cf_ib.pop(p, None)

print join
print "new cb: " + str(len(p_cb))
print "new cf: " + str(len (p_cf_ib))

831
859
new cb: 363
new cf: 217


In [44]:
p_cb_old_s1 = copy.deepcopy(p_cb)
p_cf_ib_old_s1 = copy.deepcopy(p_cf_ib)

In [33]:
print len(p_cb_old_s1)
print len(p_cf_ib_old_s1)



608
373


In [45]:
import pickle

pickle.dump(p_cb_old_s1, open( "./DumpData/top_playlists_cb_3pass", "wb" ))
pickle.dump(p_cf_ib_old_s1, open( "./DumpData/top_playlists_cf_ib_3pass", "wb" ))

#l1=pickle.load(open( "./DumpData/top_playlists_cb", "rb" ))
#l2=pickle.load(open( "./DumpData/top_playlists_cf_ib", "rb" ))